In [1]:
#default_exp rss_feeds 

In [2]:
#export
from operator import itemgetter

import numpy as np
import tqdm
import bs4
import feedparser
import seaborn as sns
import pandas as pd

In [3]:
%cd ..

/home/piotr/Documents/pytorch_hackathon


In [4]:
pd.set_option("display.max_colwidth", 250)

# RSS Feeds


In [5]:
#export
medium_publications = [
    'the-artificial-impostor',
    'pytorch',
    'dair.ai',
    'towards-artificial-intelligence',
    'swlh',
    '@ODSC',
    'doctrine',
    'paperswithcode'
]


medium_url_template = 'https://medium.com/feed/{}'
medium_url = medium_url_template.format(medium_publications[0])
medium_urls = [medium_url_template.format(publication) for publication in medium_publications]

In [6]:
#export
subreddits = [
    'MachineLearning',
    'deeplearning',
    'datascience',
    'cognitivelinguistics',
    'TopOfArxivSanity',
    'kaggle'
]

reddit_url_template = 'https://www.reddit.com/r/{}/.rss'
reddit_url = reddit_url_template.format(subreddits[0])
reddit_urls = [reddit_url_template.format(subreddit) for subreddit in subreddits]

In [7]:
#export


def get_article_text(article):
    article_html_content = article['content'][0]['value']
    article_text = bs4.BeautifulSoup(article_html_content).text
    return article_text


def get_feed_article_texts(feed):
    return [get_article_text(article) for article in feed['entries'] if 'content' in article.keys()]


def get_feed_article_df(feed):
    feed_df = pd.DataFrame.from_records(feed['entries'])
    feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)
    return feed_df


def add_field(df, field_name, values):
    df[field_name] = values
    return df

In [8]:
#export
paperswithcode_url = 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest' 
hackernews_url = 'https://news.ycombinator.com/rss' 
rss_feed_urls = [paperswithcode_url, hackernews_url] + medium_urls + reddit_urls

In [9]:
feed_urls = pd.read_table('data/feeds.txt', header=None).iloc[:,0].values

In [10]:
for url in rss_feed_urls:
    print(url)

https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest
https://news.ycombinator.com/rss
https://medium.com/feed/the-artificial-impostor
https://medium.com/feed/pytorch
https://medium.com/feed/dair.ai
https://medium.com/feed/towards-artificial-intelligence
https://medium.com/feed/swlh
https://medium.com/feed/@ODSC
https://medium.com/feed/doctrine
https://medium.com/feed/paperswithcode
https://www.reddit.com/r/MachineLearning/.rss
https://www.reddit.com/r/deeplearning/.rss
https://www.reddit.com/r/datascience/.rss
https://www.reddit.com/r/cognitivelinguistics/.rss
https://www.reddit.com/r/TopOfArxivSanity/.rss
https://www.reddit.com/r/kaggle/.rss


### Loading RSS feed articles to a dataframe

In [25]:
#export
from datetime import date

def get_feed_df(feed_urls):
    feeds = [
        (feed_url, feedparser.parse(feed_url))
        for feed_url in tqdm.tqdm(feed_urls)
    ]
    df = pd.concat(
        [
            add_field(get_feed_article_df(feed), 'feed', feed_url)
            for (feed_url, feed) in feeds
            if len(feed['entries']) > 0
        ]
    )
    
    df['date'] = df['updated']
    df['date'] = df['date'].fillna(df['published'])
    df['date'] = df['date'].fillna(date.today())
    
    df['date'] = df['date'].apply(lambda x: str(x)[:10] if str(x)[0].isdigit()
                            else str(x).split(' ')[1] + ' ' + str(x).split(' ')[2] + ' ' + str(x).split(' ')[3])    
    col = df['date'].unique()
    
    #return col
    return df
    
    
    
feed_df = get_feed_df(rss_feed_urls)
feed_df

#feedparser.parse('https://news.ycombinator.com/rss')

100%|██████████| 16/16 [00:25<00:00,  1.57s/it]


,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail,date
0,AP-Loss for Accurate One-Stage Object Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'AP-Loss for Accurate One-Stage Object Detection'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object'}]",https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object,"This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ranking task, and adopting the Average-Precision loss (AP-loss) for the ranking problem. <p>Code: <a href=""https://gi...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ...",https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object,False,"[{'term': 'Object detection', 'scheme': None, 'label': None}]","This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ranking task, and adopting the Average-Precision loss (AP-loss) for the ranking problem. Code: https://github.com/ccc...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-19
1,A Large-scale Open Dataset for Bandit Algorithms,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'A Large-scale Open Dataset for Bandit Algorithms'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit'}]",https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit,"The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different OPE estimators. <p>Code: <a href=""https://github.com/st-tech/zr-obp"">https://github.com/st-tech/zr-obp</a></p>","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different O...",https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit,False,NaN,The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different OPE estimators. Code: https://github.com/st-tech/zr-obp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-19
2,Oriented Object Detection in Aerial Images with Box Boundary-Aware Vectors,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Oriented Object Detection in Aerial Images with Box Boundary-Aware Vectors'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images'}]",https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images,"Oriented object detection in aerial images is a challenging task as the objects in aerial images are displayed in arbitrary directions and are usually densely packed. <p>Code: <a href=""https://github.com/yijingru/BBAVectors-Oriented-Object-Detect...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Oriented object detection in aerial images is a challenging task as the objects in aerial images are displayed in arbitrary direc...",https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images,False,"[{'term': 'Object detection in aerial images', 'scheme': None, 'label': None}]",Oriented object detection in aerial images is a challenging task as the 

In [26]:
feed_df.shape

(296, 23)

In [27]:
feed_df.head()

feed_df

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail,date
0,AP-Loss for Accurate One-Stage Object Detection,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'AP-Loss for Accurate One-Stage Object Detection'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object'}]",https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object,"This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ranking task, and adopting the Average-Precision loss (AP-loss) for the ranking problem. <p>Code: <a href=""https://gi...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ...",https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object,False,"[{'term': 'Object detection', 'scheme': None, 'label': None}]","This paper alleviates this issue by proposing a novel framework to replace the classification task in one-stage detectors with a ranking task, and adopting the Average-Precision loss (AP-loss) for the ranking problem. Code: https://github.com/ccc...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-19
1,A Large-scale Open Dataset for Bandit Algorithms,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'A Large-scale Open Dataset for Bandit Algorithms'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit'}]",https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit,"The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different OPE estimators. <p>Code: <a href=""https://github.com/st-tech/zr-obp"">https://github.com/st-tech/zr-obp</a></p>","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different O...",https://paperswithcode.com/paper/a-large-scale-open-dataset-for-bandit,False,NaN,The data contain the ground-truth about the performance of several bandit policies and enable the fair comparisons of different OPE estimators. Code: https://github.com/st-tech/zr-obp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-19
2,Oriented Object Detection in Aerial Images with Box Boundary-Aware Vectors,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Oriented Object Detection in Aerial Images with Box Boundary-Aware Vectors'}","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images'}]",https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images,"Oriented object detection in aerial images is a challenging task as the objects in aerial images are displayed in arbitrary directions and are usually densely packed. <p>Code: <a href=""https://github.com/yijingru/BBAVectors-Oriented-Object-Detect...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pwc/latest', 'value': 'Oriented object detection in aerial images is a challenging task as the objects in aerial images are displayed in arbitrary direc...",https://paperswithcode.com/paper/oriented-object-detection-in-aerial-images,False,"[{'term': 'Object detection in aerial images', 'scheme': None, 'label': None}]",Oriented object detection in aerial images is a challenging task as the 

In [28]:
pd.to_datetime(feed_df['date'])

0    2020-08-19
1    2020-08-19
2    2020-08-19
3    2020-08-19
4    2020-08-19
        ...    
20   2020-07-21
21   2020-07-19
22   2020-07-18
23   2020-07-17
24   2020-07-16
Name: date, Length: 296, dtype: datetime64[ns]